In [1]:
import pandas as pd
import numpy as np
from sqlite3 import connect
import os

Connect to database

In [2]:
conn = connect("..\\data\\housing.db")
cur = conn.cursor()

Initialize database tables

In [ ]:
%%sql
CREATE TABLE salary(
    year INT,
    state TEXT,
    median_salary REAL
)

Typical home value (Determined by Zillow): value for homes in the 35th to 65th percentile range

In [ ]:
%%sql
CREATE TABLE house_sale(
    year INT,
    state TEXT,
    typical_home_value REAL
)

In [ ]:
%%sql
CREATE TABLE  us_salary(
    year INT,
    median_salary REAL
)

In [ ]:
%%sql
CREATE TABLE us_house_sale(
    year int,
    median_sale REAL
)

30-Year Fixed Rate Mortgage Average in the United States

In [ ]:
%%sql
CREATE TABLE us_mortgage_rate(
    year INT,
    avg_rate REAL
)

Populate median salary tables in housing.db

**Data source**: [Federal Reserve Bank of St. Louis. (n.d.)](https://fred.stlouisfed.org/release/tables?eid=259515&rid=249)


In [3]:
# Gets path to files
folder_path = "..\\data\\MedianIncome"
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]

# Convert csv files to a dataframe
for file in csv_files:
    state = os.path.basename(file)[:-4]
    df = pd.read_csv(file)
    df.columns = ["date", "median_salary"]
    df["year"] = pd.to_datetime(df["date"]).dt.year
    
    # Load state information to database
    for index, row in df.iterrows():
        if state == "United States":
            cur.execute("INSERT INTO us_salary (year, median_salary) VALUES (?, ?)", (row["year"], row["median_salary"]))
        else:
            cur.execute("INSERT INTO salary (year, state, median_salary) VALUES (?, ?, ?)", (row["year"], state, row["median_salary"]))
    conn.commit()

In [4]:
%%sql
SELECT * FROM salary ORDER BY state

,year,state,median_salary
0,1984,Alabama,45510
1,1985,Alabama,46610
2,1986,Alabama,47860
3,1987,Alabama,47750
4,1988,Alabama,46550
...,...,...,...
2035,2019,Wyoming,76990
2036,2020,Wyoming,76570
2037,2021,Wyoming,79570
2038,2022,Wyoming,76000


In [5]:
%%sql
SELECT * FROM us_salary ORDER BY year

,year,median_salary
0,1984,58930
1,1985,60050
2,1986,62280
3,1987,63060
4,1988,63530
5,1989,64610
6,1990,63830
7,1991,61960
8,1992,61450
9,1993,61150


Populate house sales tables in housing.db

**Data sources**: 
* [Federal Reserve Bank of St. Louis. (n.d.)](https://fred.stlouisfed.org/series/MSPUS)
* [Zillow Housing Data](https://www.zillow.com/research/data/)

In [6]:
# Get US median sales data
df = pd.read_csv("..\\data\\MedianHouseSold\\United States.csv")
df.columns = ["date", "median_sale"]
df["year"] = pd.to_datetime(df["date"]).dt.year
df.head()

,date,median_sale,year
0,1963-01-01,17800,1963
1,1963-04-01,18000,1963
2,1963-07-01,17900,1963
3,1963-10-01,18500,1963
4,1964-01-01,18500,1964


In [7]:
# Populate US median sales
avg_df = df.groupby("year")["median_sale"].mean().reset_index()
for index, row in avg_df.iterrows():
    cur.execute("INSERT INTO us_house_sale (year, median_sale) VALUES (?, ?)", (row["year"], row["median_sale"]))
conn.commit()

In [8]:
%%sql
SELECT * FROM us_house_sale

,year,median_sale
0,1963,18050.000000
1,1964,18925.000000
2,1965,20125.000000
3,1966,21500.000000
4,1967,22750.000000
...,...,...
57,2020,328150.000000
58,2021,383000.000000
59,2022,432950.000000
60,2023,426525.000000


**Data Source**: [Zillow Research](https://www.zillow.com/research/data/)

In [9]:
# Get median sales by state data
df = pd.read_csv("..\\data\\MedianHouseSold\\MedianStates.csv")

df.drop("StateName", axis=1, inplace=True)
df

,RegionID,SizeRank,RegionName,RegionType,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,...,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30
0,9,0,California,state,188110.256109,188742.550827,189596.303528,191449.546076,193603.661365,195927.945504,...,756581.465067,758066.607493,761934.960064,766432.343460,768909.375491,770909.700185,772682.310069,774983.546890,776577.716233,778354.932718
1,54,1,Texas,state,110727.473905,110787.646075,110816.830520,110961.675508,111055.479394,111160.656385,...,299498.200167,300367.479595,301220.479617,301550.419480,301319.803394,300761.659685,300348.385031,300090.532553,299833.867370,299466.823298
2,14,2,Florida,state,106324.231241,106554.539424,106833.718848,107401.218812,108013.206369,108637.630851,...,393557.439999,394416.095660,395091.990286,395215.732860,394434.657532,393384.745904,392375.047571,391524.226250,390581.313239,389387.085235
3,43,3,New York,state,151685.625556,152225.775385,152745.756703,153864.095163,155021.175699,156272.299745,...,456505.704164,459493.312355,463145.212383,466501.849754,469200.870364,471724.290510,474811.923330,477666.313210,480077.616977,481903.562884
4,47,4,Pennsylvania,state,96826.092693,97033.196372,97227.818199,97624.427909,98031.845388,98450.560335,...,256878.955033,258186.664552,259836.099840,261304.989294,262179.387799,262512.871697,262759.825236,263141.794227,263875.785085,264751.674429
5,21,5,Illinois,state,125880.837963,125981.719743,126199.780237,126712.764673,127321.436742,128005.506495,...,254350.347516,256221.318460,258636.605706,260398.263089,261509.207676,262145.854667,262952.201015,263720.908428,264583.245848,265510.101009
6,44,6,Ohio,state,103775.989678,103839.406222,103935.580772,104218.386676,104649.595355,105184.635540,...,218130.220436,219686.230984,221682.092955,223485.610938,224703.538017,225514.071385,226092.668360,226496.094487,226977.257664,227606.641522
7,16,7,Georgia,state,122229.145262,122506.706514,122826.965845,123501.252313,124244.963429,125033.046887,...,321784.647770,323077.500410,324688.400597,326004.468075,326598.654344,326819.452318,327004.643730,327062.859776,326857.936894,326280.449977
8,36,8,North Carolina,state,126136.015432,126327.915075,126525.567853,126954.311641,127431.956046,127940.052598,...,320590.137341,321917.991716,323674.176204,325307.948524,325973.279671,326119.092998,325920.341852,325808.582656,325825.649389,325799.916366
9,30,9,Michigan,state,113471.037668,113533.534806,113762.341134,114218.122439,114890.266844,115628.483460,...,232368.687265,234186.967332,236420.761088,238023.791399,238964.598953,239259.739715,239548.203921,239795.077855,240379.301971,241007.127504


Handling Nans before loading to data base

In [10]:
# Get locations of null values
null_positions = np.where(df.isnull())

# Convert to list of row-column pairs
null_locations = list(zip(null_positions[0], null_positions[1]))
null_locations

[(13, 58),
 (35, 4),
 (35, 5),
 (35, 6),
 (35, 7),
 (35, 8),
 (35, 9),
 (35, 10),
 (35, 11),
 (35, 12),
 (35, 13),
 (35, 14),
 (35, 15),
 (35, 16),
 (35, 17),
 (35, 18),
 (35, 19),
 (35, 20),
 (35, 21),
 (35, 22),
 (35, 23),
 (35, 24),
 (35, 25),
 (35, 26),
 (35, 27),
 (35, 28),
 (35, 29),
 (35, 30),
 (37, 73),
 (38, 109),
 (43, 4),
 (43, 5),
 (43, 6),
 (43, 7),
 (43, 8),
 (43, 9),
 (43, 10),
 (43, 11),
 (43, 12),
 (43, 13),
 (43, 14),
 (43, 15),
 (43, 16),
 (43, 17),
 (43, 18),
 (43, 19),
 (43, 20),
 (43, 21),
 (43, 22),
 (43, 23),
 (43, 24),
 (43, 25),
 (43, 26),
 (43, 27),
 (43, 28),
 (43, 29),
 (43, 30),
 (43, 31),
 (43, 32),
 (43, 33),
 (43, 34),
 (43, 35),
 (43, 36),
 (43, 37),
 (43, 38),
 (43, 39),
 (43, 40),
 (43, 41),
 (43, 42),
 (43, 43),
 (43, 44),
 (43, 45),
 (43, 46),
 (43, 47),
 (43, 48),
 (43, 49),
 (43, 50),
 (43, 51),
 (43, 52),
 (43, 53),
 (43, 54),
 (43, 55),
 (43, 56),
 (43, 57),
 (43, 58),
 (43, 59),
 (43, 60),
 (43, 61),
 (43, 62),
 (43, 63),
 (43, 64),
 (45, 152)

**Analysis**: Dropping data from year 2000 - 2008 because North Dakota have missing data in the follow time span and replacing the scatter Nans with the mid-point between the last month and next month value 

In [11]:
columns_to_drop = df.columns[4:112]
df.drop(columns=columns_to_drop, inplace=True)
df

,RegionID,SizeRank,RegionName,RegionType,2009-01-31,2009-02-28,2009-03-31,2009-04-30,2009-05-31,2009-06-30,...,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30
0,9,0,California,state,326673.384340,321703.665452,317587.520095,313719.739044,309971.731117,306696.226527,...,756581.465067,758066.607493,761934.960064,766432.343460,768909.375491,770909.700185,772682.310069,774983.546890,776577.716233,778354.932718
1,54,1,Texas,state,138548.786369,138288.595685,138181.832760,138165.359625,138165.174405,138141.143837,...,299498.200167,300367.479595,301220.479617,301550.419480,301319.803394,300761.659685,300348.385031,300090.532553,299833.867370,299466.823298
2,14,2,Florida,state,175528.589873,172187.430931,169046.402108,165918.877761,162893.244626,160015.092903,...,393557.439999,394416.095660,395091.990286,395215.732860,394434.657532,393384.745904,392375.047571,391524.226250,390581.313239,389387.085235
3,43,3,New York,state,270869.339294,269549.223365,268379.026794,267382.479458,266466.523338,265471.820415,...,456505.704164,459493.312355,463145.212383,466501.849754,469200.870364,471724.290510,474811.923330,477666.313210,480077.616977,481903.562884
4,47,4,Pennsylvania,state,161338.637404,160874.799190,160645.641159,160507.622728,160267.553796,159938.242223,...,256878.955033,258186.664552,259836.099840,261304.989294,262179.387799,262512.871697,262759.825236,263141.794227,263875.785085,264751.674429
5,21,5,Illinois,state,182824.674675,180637.728101,178505.645182,177157.496512,175559.691256,174284.654491,...,254350.347516,256221.318460,258636.605706,260398.263089,261509.207676,262145.854667,262952.201015,263720.908428,264583.245848,265510.101009
6,44,6,Ohio,state,121500.438579,120686.941170,119951.212743,119432.922143,119067.189276,118746.056067,...,218130.220436,219686.230984,221682.092955,223485.610938,224703.538017,225514.071385,226092.668360,226496.094487,226977.257664,227606.641522
7,16,7,Georgia,state,152346.615346,150708.620739,149235.637001,147669.989178,146036.556592,144471.816733,...,321784.647770,323077.500410,324688.400597,326004.468075,326598.654344,326819.452318,327004.643730,327062.859776,326857.936894,326280.449977
8,36,8,North Carolina,state,164599.554611,163996.840520,163519.699954,163030.565949,162388.485324,161662.075855,...,320590.137341,321917.991716,323674.176204,325307.948524,325973.279671,326119.092998,325920.341852,325808.582656,325825.649389,325799.916366
9,30,9,Michigan,state,115086.817592,113052.262642,111080.087829,109385.544487,107898.282601,106537.295921,...,232368.687265,234186.967332,236420.761088,238023.791399,238964.598953,239259.739715,239548.203921,239795.077855,240379.301971,241007.127504


In [12]:
# Identify scattered Nans

# Get locations of null values
null_positions = np.where(df.isnull())

# Convert to list of row-column pairs
null_locations = list(zip(null_positions[0], null_positions[1]))
print(len(null_locations))
null_locations

2


[(45, 44), (47, 126)]

In [13]:
# Replacing Nans with the mid-point of last and next month
for nan_loc in null_locations:
    last_month = df.iloc[nan_loc[0], nan_loc[1] - 1]
    next_month = df.iloc[nan_loc[0], nan_loc[1] + 1]
    df.iloc[nan_loc[0], nan_loc[1]] = (last_month + next_month) / 2
    print(df.iloc[nan_loc[0], nan_loc[1]])

146659.08178092653
298478.79625795095


In [14]:
# Final check for nans

# Get locations of null values
null_positions = np.where(df.isnull())

# Convert to list of row-column pairs
null_locations = list(zip(null_positions[0], null_positions[1]))
print(len(null_locations))
df

0


,RegionID,SizeRank,RegionName,RegionType,2009-01-31,2009-02-28,2009-03-31,2009-04-30,2009-05-31,2009-06-30,...,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30
0,9,0,California,state,326673.384340,321703.665452,317587.520095,313719.739044,309971.731117,306696.226527,...,756581.465067,758066.607493,761934.960064,766432.343460,768909.375491,770909.700185,772682.310069,774983.546890,776577.716233,778354.932718
1,54,1,Texas,state,138548.786369,138288.595685,138181.832760,138165.359625,138165.174405,138141.143837,...,299498.200167,300367.479595,301220.479617,301550.419480,301319.803394,300761.659685,300348.385031,300090.532553,299833.867370,299466.823298
2,14,2,Florida,state,175528.589873,172187.430931,169046.402108,165918.877761,162893.244626,160015.092903,...,393557.439999,394416.095660,395091.990286,395215.732860,394434.657532,393384.745904,392375.047571,391524.226250,390581.313239,389387.085235
3,43,3,New York,state,270869.339294,269549.223365,268379.026794,267382.479458,266466.523338,265471.820415,...,456505.704164,459493.312355,463145.212383,466501.849754,469200.870364,471724.290510,474811.923330,477666.313210,480077.616977,481903.562884
4,47,4,Pennsylvania,state,161338.637404,160874.799190,160645.641159,160507.622728,160267.553796,159938.242223,...,256878.955033,258186.664552,259836.099840,261304.989294,262179.387799,262512.871697,262759.825236,263141.794227,263875.785085,264751.674429
5,21,5,Illinois,state,182824.674675,180637.728101,178505.645182,177157.496512,175559.691256,174284.654491,...,254350.347516,256221.318460,258636.605706,260398.263089,261509.207676,262145.854667,262952.201015,263720.908428,264583.245848,265510.101009
6,44,6,Ohio,state,121500.438579,120686.941170,119951.212743,119432.922143,119067.189276,118746.056067,...,218130.220436,219686.230984,221682.092955,223485.610938,224703.538017,225514.071385,226092.668360,226496.094487,226977.257664,227606.641522
7,16,7,Georgia,state,152346.615346,150708.620739,149235.637001,147669.989178,146036.556592,144471.816733,...,321784.647770,323077.500410,324688.400597,326004.468075,326598.654344,326819.452318,327004.643730,327062.859776,326857.936894,326280.449977
8,36,8,North Carolina,state,164599.554611,163996.840520,163519.699954,163030.565949,162388.485324,161662.075855,...,320590.137341,321917.991716,323674.176204,325307.948524,325973.279671,326119.092998,325920.341852,325808.582656,325825.649389,325799.916366
9,30,9,Michigan,state,115086.817592,113052.262642,111080.087829,109385.544487,107898.282601,106537.295921,...,232368.687265,234186.967332,236420.761088,238023.791399,238964.598953,239259.739715,239548.203921,239795.077855,240379.301971,241007.127504


In [15]:
# Reformating data frame
data = []
dates = sorted(list(set(df.columns) - {"RegionID", "SizeRank", "RegionName", "RegionType", "StateName"}))

for index, row in df.iterrows():
    for col_date in dates:
        year, month, day = col_date.split("-")
        info = {"state": row["RegionName"], 
                "year": int(year),
                "value": df.loc[index, col_date]}
        data.append(info)
normalized_df = pd.DataFrame(data)
normalized_df

,state,year,value
0,California,2009,326673.384340
1,California,2009,321703.665452
2,California,2009,317587.520095
3,California,2009,313719.739044
4,California,2009,309971.731117
...,...,...,...
9736,Wyoming,2024,343370.401509
9737,Wyoming,2024,344757.832431
9738,Wyoming,2024,347033.423656
9739,Wyoming,2024,349781.067684


In [16]:
# Group data frame to only have one year 
grouped_df = normalized_df.groupby(["state", "year"])["value"].mean().reset_index()
for index, row in grouped_df.iterrows():
    cur.execute("INSERT INTO house_sale (year, state, typical_home_value) VALUES (?, ?, ?)", (row["year"], row["state"], row["value"]))
conn.commit()
grouped_df

,state,year,value
0,Alabama,2009,128447.614467
1,Alabama,2010,123302.552877
2,Alabama,2011,117603.501168
3,Alabama,2012,119353.419808
4,Alabama,2013,126528.258275
...,...,...,...
811,Wyoming,2020,263432.215999
812,Wyoming,2021,289975.703000
813,Wyoming,2022,321046.654803
814,Wyoming,2023,332973.677888


In [17]:
%%sql
SELECT * FROM house_sale ORDER BY year

,year,state,typical_home_value
0,2009,Alabama,128447.614467
1,2009,Alaska,248788.519082
2,2009,Arizona,173044.572368
3,2009,Arkansas,119439.742325
4,2009,California,308873.463094
...,...,...,...
811,2024,Virginia,384555.898690
812,2024,Washington,580319.010980
813,2024,West Virginia,162207.137012
814,2024,Wisconsin,297702.544983


**Data Source**: [Federal Reserve Bank of St. Louis. (n.d.)](https://fred.stlouisfed.org/series/MORTGAGE30US)

In [18]:
df = pd.read_csv("..\\data\\mortgages\\MORTGAGE30US.csv")
df.columns = ["date", "rate"]

df["year"] = pd.to_datetime(df["date"]).dt.year
df["rate"] = df["rate"] / 100
df = df.groupby("year")["rate"].mean().reset_index()

for index, row in df.iterrows():
    cur.execute("INSERT INTO us_mortgage_rate (year, avg_rate) VALUES (?, ?)", (row["year"], row["rate"]))
conn.commit()
df

,year,rate
0,1971,0.075417
1,1972,0.073833
2,1973,0.080448
3,1974,0.091871
4,1975,0.090471
5,1976,0.088658
6,1977,0.088452
7,1978,0.096417
8,1979,0.112037
9,1980,0.137421


In [19]:
%%sql
SELECT * FROM us_mortgage_rate

,year,avg_rate
0,1971,0.075417
1,1972,0.073833
2,1973,0.080448
3,1974,0.091871
4,1975,0.090471
5,1976,0.088658
6,1977,0.088452
7,1978,0.096417
8,1979,0.112037
9,1980,0.137421
